---

# IT00CJ42: Search and Optimization Algorithms

**Restart the kernel and run all cells** before you turn this problem in, make sure everything runs as expected.

Make sure you fill in any place that says `YOUR CODE HERE`.

---

## Miniproject 3: Mobile Application for Power Consumption

We want to find an optimal configuration to run the EEMBC CoreMark-Pro Suite 1 benchmark suite on the ODROID XU3 development board from HARDKERNEL from the point of view of power consumption.

The ODROID XU3 board includes a Exynos 5422 processor that implements the ARM big.LITTLE architecture with two clusters composed of 4 cores each. The big cluster consists of a high-performance Cortex-A15 quad-core CPUs, and the little cluster is a low power Cortex-A7 quad-core CPUs.

![odroid](https://cdn.hardkernel.com/wp-content/uploads/2018/10/ODROID-XU3.jpg)

As search space, we consider the different board configurations in terms of the number of CPUs, type of CPU, core performance level (i.e., Dynamic voltage and frequency scaling, DVFS level) and core utilization level.

The core utilization level is set using a scheduler framework, which is based on the concept of setting the CPU bandwidth of a certainprocess. By setting the runtime of a process inside a period we can control the level of CPU it uses. In this experiment, we considered 10 possible core utilization levels from 10% to 100% with 10% steps.

The search  space has therefore 6 dimensions and is defined  as follows:
* p1 Core 1 mode, enumeration, valid range [0,7]
* p2 Core 1 frequency, Hz, valid range [200, 1400]
* p3 Core 1 max. utilization, %, valid range [0, 100]
* p4 Core 2 mode, enumeration, valid range [1,4]
* p5 Core 2 frequency, Hz, valid range [200, 2000]
* p6 Core 1 max. utilization, %, valid range [0, 100]

For each configuration, we obtain 3 different performance indicators:
* Power (Watt), lower is better
* Performance  (Instructions/seconds), higher is better
* Efficiency      (Instructions/Joule), higher is better

There are 479 600 different configurations and executing the benchmark through the entire search space requires around 80 hours.

**Your task is to create a Python program to find  the best configurations for the Odroid board. Since there are three performance indicators, this is a multi-objective optimization problem.** <br>
**To simplify this project, we have already collected the benchmark results for all the configurations. They are stored in the file odroid.npy. It can be accesed using the function evaluate_performance:**

### Imports

In [5]:
import numpy as np

### Solutions check
We use the function **check** to implement tests for your solution

In [6]:
def check(expression, message=""):
    if not expression:
        raise AssertionError(message)
    return "Passed"

### Example

In [7]:
def evaluate_performance(p1, p2, p3, p4, p5, p6, data):
    """
    Estimate the performance of the odroid board when running the benchmark
    :param p1: Core 1 mode, enumeration, valid range [0,7]
    :param p2: Core 1 frequency, Hz, valid range [200, 1400]
    :param p3: Core 1 max. utilization, %, valid range [0, 100]
    :param p4: Core 2 mode, enumeration, valid range [1,4]
    :param p5: Core 2 frequency, Hz, valid range [200, 2000]
    :param p6: Core 1 max. utilization, %, valid range [0, 100]
    :param data: A numpy array contianing experimental data about the bechmark
    :return: A list with 2 performance indicators:
        Power(W) and Performance(Instructions/s)
    """
    parameters = data[:, 0: 6]
    performance = data[:, 6:]

    # look up the experimental data that is closer to the provided parameters
    distances = np.sum((parameters - np.asarray([p1, p2, p3, p4, p5, p6])) ** 2, axis=1)
    return performance[np.argmin(distances)][0:2]

#### Load the odroid experimental data

In [8]:
odroid_data = np.load("../res/data/odroid.npy")

#### Query a configurarion

In [9]:
result = evaluate_performance(1, 250, 100, 2, 1500, 75, odroid_data)

print(f'Power: {result[0]}\nPerformance: {result[1]}')

Power: 2.2981752663802366
Performance: 2122866971.5419734


### Solution
Implement a multi-objective optimization algorithm and compute a pareto set for power and performance for the odroid dataset. <br>
Meaning:
* You should implement a multi-objective optimization algorithm, that uses power and performance as fitness functions
* For finding a set of six (6) optimal parameters (p1-p6) to optimize the system under test (odroid)
* Use the function *evaluate_performance* to evaluate the power and performance of the system (for example using a weighed sum)
* Check your best performing set of parameters against the check

In [10]:
# Randomly generate a set of configurations within specified parameter ranges
sample_size = 1000
configs = np.random.randint(low=[0, 200, 10, 1, 200, 10], high=[8, 1401, 101, 5, 2001, 101], size=(sample_size, 6))

def find_optimal_configurations(data):
    # Set a random seed for reproducibility
    np.random.seed(42)
        
    # Initialize variables to track the optimal power and performance found
    optimal_power = float('inf')  # Start with infinity, as we're looking for the minimum
    optimal_performance = 0  # Start with zero, as we're looking for the maximum

    # Iterate over each configuration in the sample
    for config in configs:
        # Evaluate the current configuration to get its power and performance
        power, performance = evaluate_performance(*config, data)
        
        # Check if the current configuration satisfies both conditions
        if power <= 8 and performance >= 100 * 100000000:
            optimal_power = power
            optimal_performance = performance
            optimal_config = config
    
    return optimal_config.tolist(), [optimal_power, optimal_performance]

optimal_parameters, result = find_optimal_configurations(odroid_data)
print(f"Optimal Parameters: {optimal_parameters}")
print(f"Optimal Power: {result[0]}")
print(f"Optimal Performance: {result[1]}")

Optimal Parameters: [6, 612, 98, 2, 1846, 96]
Optimal Power: 5.332701515717865
Optimal Performance: 10871428415.05537


In [11]:
# check power
check(result[0] <= 8, "Power consumption is too high.")

'Passed'

In [12]:
# check performance
check(result[1] >= 100*100000000, "Performance is too low.")

'Passed'

In [13]:
# check both
check(result[0] <= 8 and result[1] >= 100*100000000, "Power consumption is too high AND/OR performance is too low.")

'Passed'